In [81]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [82]:
batch_size = 1
num_classes = 10
learnin_rate = 0.001
num_epochs = 20

In [83]:
dataset = torch.load('dataset/images.pth')
labels = torch.load('dataset/labels.pth')

In [84]:
class handDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.x = dataset
        self.y = labels
        self.n_samples = len(dataset)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [85]:
generator = torch.Generator().manual_seed(42)

train_size = int(len(dataset) * .7)
test_size = len(dataset) - train_size

hand_dataset = handDataset()
train_dataset, test_dataset = torch.utils.data.random_split(hand_dataset, [train_size, test_size], generator=generator)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [86]:
class FF_model(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64*64*3,500) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(500, num_classes)  
    
    def forward(self, x):
        out = self.flatten(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [87]:
class CNN_model(nn.Module):

    def __init__(self, num_classes):
        super(CNN_model, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(14*14*64,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        print(x.shape)

        out = self.conv_layer1(x)
        out = self.max_pool1(out)

        print(out.shape)
        
        out = self.conv_layer2(out)
        out = self.max_pool2(out)

        print(out.shape)

        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)

        return out


In [88]:
cnn_model = CNN_model(num_classes)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(cnn_model.parameters(), lr=learnin_rate, weight_decay=0.005, momentum=0.9)

total_step = len(train_loader)

In [89]:
for epoch in range(num_epochs):

    for i, (images, labels) in enumerate(train_loader):
        
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

torch.Size([1, 3, 64, 64])
torch.Size([1, 32, 30, 30])
torch.Size([1, 64, 14, 14])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (896x14 and 12544x128)

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))